In [1]:
# Loading data

import pandas as pd
import numpy as np

data = pd.read_stata('maha.dta')

In [2]:
# Printing to check if data loaded properly

data.head()

,HHID,CentreCodeRndShift,Vill_Blk_Slno,Round,Schedule,Sample,Sector,ruralurban,State_Region,State,...,OOPshare,rural_OOPshare,urban_OOPshare,BPL_share,bpl_MPCE,Pre,ten_hce,cat,cat_code,Pi
0,365962301,005,36596,71,250,1,1,1,272,27,...,0.000000,0.000000,NaN,NaN,NaN,NaN,0.004386,NaN,0.0,0.0
1,361661204,003,36166,71,250,1,1,1,274,27,...,0.000000,0.000000,NaN,NaN,NaN,NaN,0.013889,NaN,0.0,0.0
2,363871101,007,36387,71,250,1,1,1,273,27,...,0.000000,0.000000,NaN,0.0,400.0,-567.109985,0.020833,NaN,0.0,1.0
3,366352101,003,36635,71,250,1,1,1,272,27,...,0.047059,0.047059,NaN,NaN,NaN,NaN,0.009804,4800.0,1.0,0.0
4,361892201,007,36189,71,250,1,1,1,274,27,...,0.000000,0.000000,NaN,NaN,NaN,NaN,0.010417,NaN,0.0,0.0


In [3]:
data.loc[0,'hhsize']

9

In [4]:
# before_hp = Flag variable to indicate APL/BPL of household before health payment, Not seggregated by rural or urban
# rural_afthp = Flag variable to indicate APL/BPL of household after health payment in rural area
# urban_afthp = Flag variable to indicate APL/BPL of household after health payment in urban area

In [5]:
data['before_hp'].value_counts()

0.0    21739
1.0     5385
Name: before_hp, dtype: int64

In [6]:
data['rural_aftHP'].value_counts()

0.0    10317
1.0     3755
Name: rural_aftHP, dtype: int64

In [7]:
data['urban_aftHP'].value_counts() # before 5385, after 5943

0.0    10864
1.0     2188
Name: urban_aftHP, dtype: int64

In [8]:
data.loc[0,'before_hp']

0.0

In [9]:
data['State_District'].unique()

array(['2725', '2719', '2701', '2731', '2727', '2735', '2714', '2730',
       '2732', '2708', '2718', '2721', '2703', '2734', '2702', '2726',
       '2713', '2712', '2709', '2711', '2717', '2715', '2724', '2728',
       '2729', '2706', '2707', '2705', '2704', '2720', '2716', '2710',
       '2733', '2722'], dtype=object)

In [10]:
# Checking length of total data loaded

len(data)

27124

In [11]:
# Decoding Sector variable
cleanup_nums = {"Sector":     {"2": "Urban", "1": "Rural"}}
data.replace(cleanup_nums, inplace=True)
data['Sector']
#                "Sample": {"2": "State", "1": "Central"}

0        Rural
1        Rural
2        Rural
3        Rural
4        Rural
5        Rural
6        Rural
7        Rural
8        Rural
9        Rural
10       Rural
11       Rural
12       Rural
13       Rural
14       Rural
15       Rural
16       Rural
17       Rural
18       Rural
19       Rural
20       Rural
21       Rural
22       Rural
23       Rural
24       Rural
25       Rural
26       Rural
27       Rural
28       Rural
29       Rural
         ...  
27094    Urban
27095    Urban
27096    Urban
27097    Urban
27098    Urban
27099    Urban
27100    Urban
27101    Urban
27102    Urban
27103    Urban
27104    Urban
27105    Urban
27106    Urban
27107    Urban
27108    Urban
27109    Urban
27110    Urban
27111    Urban
27112    Urban
27113    Urban
27114    Urban
27115    Urban
27116    Urban
27117    Urban
27118    Urban
27119    Urban
27120    Urban
27121    Urban
27122    Urban
27123    Urban
Name: Sector, Length: 27124, dtype: object

In [12]:
# Filtering Urban only data

urban_data = data[data['Sector']=="Urban"]
urban_data.head()

,HHID,CentreCodeRndShift,Vill_Blk_Slno,Round,Schedule,Sample,Sector,ruralurban,State_Region,State,...,OOPshare,rural_OOPshare,urban_OOPshare,BPL_share,bpl_MPCE,Pre,ten_hce,cat,cat_code,Pi
14072,156261101,002,15626,71,250,1,Urban,2,275,27,...,0.0,NaN,0.0,NaN,NaN,NaN,0.013889,NaN,0.0,0.0
14073,155681101,008,15568,71,250,1,Urban,2,271,27,...,0.0,NaN,0.0,NaN,NaN,NaN,0.003968,NaN,0.0,0.0
14074,155711302,004,15571,71,250,1,Urban,2,274,27,...,0.0,NaN,0.0,NaN,NaN,NaN,0.006410,NaN,0.0,0.0
14075,155681203,004,15568,71,250,1,Urban,2,271,27,...,0.0,NaN,0.0,NaN,NaN,NaN,0.004167,NaN,0.0,0.0
14076,155481204,003,15548,71,250,1,Urban,2,271,27,...,0.0,NaN,0.0,NaN,NaN,NaN,0.004167,NaN,0.0,0.0


In [13]:
# Checking if PL threshold values in MPCE column

if 967.11 in data['MPCE'] or 1126.48 in data['MPCE']:
    print(True)
else:
    print(False)

False


In [14]:
# Creating a PL_flag column which indicates whether APL or BPL depending upon MPCE

data['PL_flag'] = 0

for i in range(len(data)):
    if data.loc[(i,'Sector')]=='Rural':
        if data.loc[(i,'MPCE')]>967.11:
            data.loc[(i,'PL_flag')] = "APL"
        else:
            data.loc[(i,'PL_flag')] = "BPL"
    elif data.loc[(i,'Sector')]=='Urban':
        if data.loc[(i,'MPCE')]>1126.48:
            data.loc[(i,'PL_flag')] = "APL"
        else:
            data.loc[(i,'PL_flag')] = "BPL"

In [15]:
# Checking for new column Pl_flag

data.head()

,HHID,CentreCodeRndShift,Vill_Blk_Slno,Round,Schedule,Sample,Sector,ruralurban,State_Region,State,...,rural_OOPshare,urban_OOPshare,BPL_share,bpl_MPCE,Pre,ten_hce,cat,cat_code,Pi,PL_flag
0,365962301,005,36596,71,250,1,Rural,1,272,27,...,0.000000,NaN,NaN,NaN,NaN,0.004386,NaN,0.0,0.0,APL
1,361661204,003,36166,71,250,1,Rural,1,274,27,...,0.000000,NaN,NaN,NaN,NaN,0.013889,NaN,0.0,0.0,APL
2,363871101,007,36387,71,250,1,Rural,1,273,27,...,0.000000,NaN,0.0,400.0,-567.109985,0.020833,NaN,0.0,1.0,BPL
3,366352101,003,36635,71,250,1,Rural,1,272,27,...,0.047059,NaN,NaN,NaN,NaN,0.009804,4800.0,1.0,0.0,APL
4,361892201,007,36189,71,250,1,Rural,1,274,27,...,0.000000,NaN,NaN,NaN,NaN,0.010417,NaN,0.0,0.0,APL


In [16]:
# Table 1

# Step 1 - Filtering the dataset to look at only those records where cat_code = 1
tab1_data = data[data['cat_code']==1]
tab1_data.head()

,HHID,CentreCodeRndShift,Vill_Blk_Slno,Round,Schedule,Sample,Sector,ruralurban,State_Region,State,...,rural_OOPshare,urban_OOPshare,BPL_share,bpl_MPCE,Pre,ten_hce,cat,cat_code,Pi,PL_flag
3,366352101,003,36635,71,250,1,Rural,1,272,27,...,0.047059,NaN,NaN,NaN,NaN,0.009804,4800.000000,1.0,0.0,APL
6,366532201,005,36653,71,250,1,Rural,1,272,27,...,0.013725,NaN,NaN,NaN,NaN,0.009804,1400.000000,1.0,0.0,APL
14,363802301,008,36380,71,250,1,Rural,1,273,27,...,0.002623,NaN,NaN,NaN,NaN,0.004630,566.666687,1.0,0.0,APL
15,366162201,004,36616,71,250,1,Rural,1,272,27,...,0.003488,NaN,NaN,NaN,NaN,0.016026,217.666672,1.0,0.0,APL
34,366831202,005,36683,71,250,1,Rural,1,274,27,...,0.034434,NaN,NaN,NaN,NaN,0.010163,3388.333252,1.0,0.0,APL


In [17]:
# Step 2 - Creating District code column

tab1_data['District_Code'] = tab1_data['State_District'].str.slice(start=2)

C:\Users\sathw\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
tab1_data.head()

,HHID,CentreCodeRndShift,Vill_Blk_Slno,Round,Schedule,Sample,Sector,ruralurban,State_Region,State,...,urban_OOPshare,BPL_share,bpl_MPCE,Pre,ten_hce,cat,cat_code,Pi,PL_flag,District_Code
3,366352101,003,36635,71,250,1,Rural,1,272,27,...,NaN,NaN,NaN,NaN,0.009804,4800.000000,1.0,0.0,APL,31
6,366532201,005,36653,71,250,1,Rural,1,272,27,...,NaN,NaN,NaN,NaN,0.009804,1400.000000,1.0,0.0,APL,35
14,363802301,008,36380,71,250,1,Rural,1,273,27,...,NaN,NaN,NaN,NaN,0.004630,566.666687,1.0,0.0,APL,03
15,366162201,004,36616,71,250,1,Rural,1,272,27,...,NaN,NaN,NaN,NaN,0.016026,217.666672,1.0,0.0,APL,34
34,366831202,005,36683,71,250,1,Rural,1,274,27,...,NaN,NaN,NaN,NaN,0.010163,3388.333252,1.0,0.0,APL,29


In [19]:
# Creating the crosstable

pd.crosstab(tab1_data.District_Code, [tab1_data.Sector, tab1_data.PL_flag])

Sector        Rural     Urban    
PL_flag         APL BPL   APL BPL
District_Code                    
01               11  19     8   9
02               28   9    10   4
03               73  27    35  22
04               66  25    21   7
05               27   8    27   9
06               39   7    14   1
07               29  34    37  21
08                8   5    17   4
09               29  15    87  16
10               14   7     8   5
11                6  16    21   4
12               10   6     9   7
13               25  24    15  10
14               30  42    31   3
15               47   2    28   0
16               62   0    15   0
17               44   1    40   2
18               29   3     6   7
19               18   1    35   0
20              115   7   122   4
21               90  17   166  19
22                0   0   214   9
24               56   9    11   0
25               58  10   184   3
26               62   4    27   3
27               29   1    20   1
28               38   7    29   0
29               73   7    19   2
30               87  15    74   7
31               51   3    22   3
32               50   2    11   0
33               18   9     5   1
34               89  21    34   7
35               70   9    34   4

In [20]:
len(tab1_data)

3483

In [21]:
# Creating a P_Gap column which is the difference between MPCE and threshold Poverty value in Rural/Urban areas

data['P_Gap'] = 0
data['Po_Gap'] = 0

for i in range(len(data)):
    if data.loc[(i,'Sector')]=='Rural':
        data.loc[(i,'P_Gap')] = 967.11 - data.loc[(i,'MPCE')]
        data.loc[(i,'Po_Gap')] = 967.11 - (data.loc[(i,'MPCE')]-data.loc[(i,'tot_oop')])
    elif data.loc[(i,'Sector')]=='Urban':
        data.loc[(i,'P_Gap')] = 1126.48 - data.loc[(i,'MPCE')]
        data.loc[(i,'Po_Gap')] = 1126.48 - (data.loc[(i,'MPCE')]-data.loc[(i,'tot_oop')])

In [22]:
# Pre G is MPCE - threshold
# Post G is 

In [23]:
data['P_Gap']

0       -1144.001084
1         -32.890000
2         567.110000
3        -247.175767
4        -366.223374
5        -216.223374
6        -247.175767
7         342.110000
8        -182.890000
9         267.110000
10       -245.390000
11       -532.890000
12       -632.890000
13        -14.708176
14      -1604.318467
15        -72.890000
16       -866.223374
17        -32.890000
18        279.610000
19        167.110000
20        250.443313
21        522.665542
22         67.110000
23      -1032.890000
24        442.110000
25      -1132.890000
26       -366.223374
27         17.110000
28      -1255.112168
29       -532.890000
            ...     
27094   -2273.520000
27095     563.980000
27096   -1498.520000
27097   -1317.964336
27098   -2206.853252
27099     126.480000
27100   -4273.520000
27101    -873.520000
27102    -302.091411
27103    -206.853374
27104   -1273.520000
27105   -2873.520000
27106   -1373.520000
27107   -1573.520000
27108   -1373.520000
27109     459.813313
27110   -1873

In [24]:
# cleaning rural_aftHP and urban_aftHP columns

# data['urban_aftHP'].fillna(value=2.0,inplace=True)

In [25]:
# data['urban_aftHP'].unique()

In [26]:
# cleaning rural_aftHP and urban_aftHP columns

# data['rural_aftHP'].fillna(value=2.0,inplace=True)
# data['rural_aftHP'].unique()

In [27]:
# Checking if before_hp, after_hp etc. are disjoint/mutually exclusive

# for i in range(len(data)):
#     count=0
#     if data.loc[(i,'before_hp')]+data.loc[(i,'urban_aftHP')]+data.loc[(i,'rural_aftHP')]>3.0:
#         print(False)
#         print(data.loc[(i,'before_hp')],data.loc[(i,'urban_aftHP')],data.loc[(i,'rural_aftHP')])
#         break
#     else:
#         count+=1

In [28]:
# Crosstab will not work for Table 2, hence there is overlap among before_hp and after_hp values. 
# i.e. a single row will feature in both calculations

In [29]:
# Need to create variables which tells APL or BPL pre and post hp. These variables will be created from the before_hp and aft_hp columns

cleanup_nums2 = {"before_hp":     {1.0: "BPL", 0.0: "APL"}}
data.replace(cleanup_nums2, inplace=True)
data['before_hp']

0        APL
1        APL
2        BPL
3        APL
4        APL
5        APL
6        APL
7        BPL
8        APL
9        BPL
10       APL
11       APL
12       APL
13       APL
14       APL
15       APL
16       APL
17       APL
18       BPL
19       BPL
20       BPL
21       BPL
22       BPL
23       APL
24       BPL
25       APL
26       APL
27       BPL
28       APL
29       APL
        ... 
27094    APL
27095    BPL
27096    APL
27097    APL
27098    APL
27099    BPL
27100    APL
27101    APL
27102    APL
27103    APL
27104    APL
27105    APL
27106    APL
27107    APL
27108    APL
27109    BPL
27110    APL
27111    APL
27112    APL
27113    APL
27114    APL
27115    BPL
27116    BPL
27117    APL
27118    APL
27119    APL
27120    APL
27121    APL
27122    APL
27123    BPL
Name: before_hp, Length: 27124, dtype: object

In [30]:
# Adding quintile to APL

for i in range(len(data)):
    if data.loc[(i,'before_hp')]=="APL":
        data.loc[(i,'before_hp')]+=str(int(data.loc[(i,'quintile')]))
        
data['before_hp']

0        APL5
1        APL1
2         BPL
3        APL2
4        APL2
5        APL2
6        APL2
7         BPL
8        APL3
9         BPL
10       APL1
11       APL4
12       APL4
13       APL3
14       APL5
15       APL1
16       APL3
17       APL2
18        BPL
19        BPL
20        BPL
21        BPL
22        BPL
23       APL2
24        BPL
25       APL3
26       APL2
27        BPL
28       APL5
29       APL1
         ... 
27094    APL5
27095     BPL
27096    APL3
27097    APL5
27098    APL5
27099     BPL
27100    APL5
27101    APL3
27102    APL3
27103    APL1
27104    APL3
27105    APL4
27106    APL4
27107    APL3
27108    APL4
27109     BPL
27110    APL4
27111    APL5
27112    APL2
27113    APL3
27114    APL3
27115     BPL
27116     BPL
27117    APL1
27118    APL4
27119    APL1
27120    APL2
27121    APL4
27122    APL2
27123     BPL
Name: before_hp, Length: 27124, dtype: object

In [31]:
data['after_hp'] = 0

for i in range(len(data)):
    if data.loc[(i,'Sector')]=='Rural':
        if data.loc[(i,'rural_aftHP')]==1.0:
            data.loc[(i,'after_hp')] = "BPL"
        elif data.loc[(i,'rural_aftHP')]==0.0:
            data.loc[(i,'after_hp')] = "APL"+str(int(data.loc[(i,'quintile')]))
    elif data.loc[(i,'Sector')]=='Urban':
        if data.loc[(i,'urban_aftHP')]==1.0:
            data.loc[(i,'after_hp')] = "BPL"
        elif data.loc[(i,'urban_aftHP')]==0.0:
            data.loc[(i,'after_hp')] = "APL"+str(int(data.loc[(i,'quintile')]))

In [32]:
data['after_hp']

0        APL5
1        APL1
2         BPL
3         BPL
4        APL2
5        APL2
6        APL2
7         BPL
8        APL3
9         BPL
10       APL1
11       APL4
12       APL4
13       APL3
14       APL5
15       APL1
16       APL3
17       APL2
18        BPL
19        BPL
20        BPL
21        BPL
22        BPL
23       APL2
24        BPL
25       APL3
26       APL2
27        BPL
28       APL5
29       APL1
         ... 
27094    APL5
27095     BPL
27096    APL3
27097    APL5
27098    APL5
27099     BPL
27100    APL5
27101    APL3
27102    APL3
27103    APL1
27104    APL3
27105    APL4
27106    APL4
27107    APL3
27108    APL4
27109     BPL
27110    APL4
27111    APL5
27112    APL2
27113    APL3
27114    APL3
27115     BPL
27116     BPL
27117    APL1
27118    APL4
27119    APL1
27120    APL2
27121    APL4
27122    APL2
27123     BPL
Name: after_hp, Length: 27124, dtype: object

In [33]:
# Function to calculate each value - Rural (R), Urban (U), Combined (C)

def calculus(data, sector, status, prepost):
    total = 0
    count = 0
    for i in range(len(data)):
        if data.loc[(i,'Sector')]==sector and prepost=="Pre":
            if data.loc[(i,'before_hp')]==status:
                total+=data.loc[(i,'P_Gap')]
                count+=1
        elif data.loc[(i,'Sector')]==sector and prepost=="Post":
            if data.loc[(i,'after_hp')]==status:
                total+=data.loc[(i,'Po_Gap')]
                count+=1
        elif sector=="Combined" and prepost=="Pre":
            if data.loc[(i,'before_hp')]==status:
                total+=data.loc[(i,'P_Gap')]
                count+=1
        elif sector=="Combined" and prepost=="Post":
            if data.loc[(i,'after_hp')]==status:
                total+=data.loc[(i,'Po_Gap')]
                count+=1
    return round((total/count),2), count          

In [47]:
print(calculus(data, "Rural", "BPL", "Pre"),"\n",
      calculus(data, "Rural", "BPL", "Post"),"\n",
      calculus(data, "Urban", "BPL", "Pre"),"\n",
      calculus(data, "Urban", "BPL", "Post"),"\n",
      calculus(data, "Combined", "BPL", "Pre"),"\n",
      calculus(data, "Combined", "BPL", "Post"),"\n",
      calculus(data, "Rural", "APL1", "Pre"),"\n",
      calculus(data, "Rural", "APL1", "Post"),"\n",
      calculus(data, "Urban", "APL1", "Pre"),"\n",
      calculus(data, "Urban", "APL1", "Post"),"\n",
      calculus(data, "Combined", "APL1", "Pre"),"\n",
      calculus(data, "Combined", "APL1", "Post"),"\n",
      calculus(data, "Rural", "APL2", "Pre"),"\n",
      calculus(data, "Rural", "APL2", "Post"),"\n",
      calculus(data, "Urban", "APL2", "Pre"),"\n",
      calculus(data, "Urban", "APL2", "Post"),"\n",
      calculus(data, "Combined", "APL2", "Pre"),"\n",
      calculus(data, "Combined", "APL2", "Post"),"\n",
      calculus(data, "Rural", "APL3", "Pre"),"\n",
      calculus(data, "Rural", "APL3", "Post"),"\n",
      calculus(data, "Urban", "APL3", "Pre"),"\n",
      calculus(data, "Urban", "APL3", "Post"),"\n",
      calculus(data, "Combined", "APL3", "Pre"),"\n",
      calculus(data, "Combined", "APL3", "Post"),"\n",
      calculus(data, "Rural", "APL4", "Pre"),"\n",
      calculus(data, "Rural", "APL4", "Post"),"\n",
      calculus(data, "Urban", "APL4", "Pre"),"\n",
      calculus(data, "Urban", "APL4", "Post"),"\n",
      calculus(data, "Combined", "APL4", "Pre"),"\n",
      calculus(data, "Combined", "APL4", "Post"),"\n",
      calculus(data, "Rural", "APL5", "Pre"),"\n",
      calculus(data, "Rural", "APL5", "Post"),"\n",
      calculus(data, "Urban", "APL5", "Pre"),"\n",
      calculus(data, "Urban", "APL5", "Post"),"\n",
      calculus(data, "Combined", "APL5", "Pre"),"\n",
      calculus(data, "Combined", "APL5", "Post"))

(230.62, 3398) 
 (678.72, 3755) 
 (254.84, 1987) 
 (992.91, 2188) 
 (239.56, 5385) 
 (794.39, 5943) 
 (-367.26, 3075) 
 (-321.92, 2862) 
 (-566.81, 1292) 
 (-526.74, 1220) 
 (-426.3, 4367) 
 (-383.13, 4082) 
 (-529.32, 3074) 
 (-426.11, 2984) 
 (-665.69, 1967) 
 (-599.07, 1898) 
 (-582.53, 5041) 
 (-493.35, 4882) 
 (-698.49, 2181) 
 (-568.13, 2145) 
 (-1087.48, 2445) 
 (-959.57, 2407) 
 (-904.09, 4626) 
 (-775.11, 4552) 
 (-1046.5, 1318) 
 (-852.38, 1309) 
 (-1687.15, 2290) 
 (-1481.06, 2276) 
 (-1453.12, 3608) 
 (-1251.51, 3585) 
 (-1696.88, 1026) 
 (-1437.18, 1017) 
 (-3324.9, 3071) 
 (-3043.08, 3063) 
 (-2917.2, 4097) 
 (-2642.78, 4080)
